## Описание проекта

Нужно защитить данные клиентов страховой компании. Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуовать корректность его работы.
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 

### Пункт 1

Импортируем необходимые библиотеки, модели и метрики

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

Загружаем датасет

In [2]:
import os

pth1 = 'insurance.csv'
pth2 = '/datasets/insurance.csv'


if os.path.exists(pth1):
    df = pd.read_csv(pth1)
    
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
    
else:
    print('Something is wrong')

Изучаем данные

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
df.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Для удобства меняем названия столбцов в датасете

In [5]:
df = df.rename(columns = {'Пол' : 'gender', 'Возраст' : 'age', 'Зарплата' : 'salary', 'Члены семьи' : 'family_members', 
                'Страховые выплаты' : 'insurance_payments'})
df.head(5)

,gender,age,salary,family_members,insurance_payments
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Меняем формат чисел на целочисленный в столбцах: <font color=red>'age', 'salary'</font>

In [6]:
df = df.astype({'age': np.int64, 'salary': np.int64})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   gender              5000 non-null   int64
 1   age                 5000 non-null   int64
 2   salary              5000 non-null   int64
 3   family_members      5000 non-null   int64
 4   insurance_payments  5000 non-null   int64
dtypes: int64(5)
memory usage: 195.4 KB


Проверяем:

In [7]:
df.head(5)

,gender,age,salary,family_members,insurance_payments
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


Проверяем имеются ли дубликаты:

In [8]:
df.duplicated().sum()

153

Удаляем дубликаты и проверяем еще раз датасет на наичие дубоикатов:

In [9]:
df = df.drop_duplicates()

In [10]:
df.duplicated().sum()

0

Дубликатов нет

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   gender              4847 non-null   int64
 1   age                 4847 non-null   int64
 2   salary              4847 non-null   int64
 3   family_members      4847 non-null   int64
 4   insurance_payments  4847 non-null   int64
dtypes: int64(5)
memory usage: 227.2 KB


Проверяем уикальные значения в столбцах: <font color=red> 'family_members' , 'insurance_payments' </font>

In [12]:
df['family_members'].unique()

array([1, 0, 2, 4, 3, 5, 6], dtype=int64)

In [13]:
df['insurance_payments'].unique()

array([0, 1, 2, 3, 5, 4], dtype=int64)

In [14]:
df.describe(percentiles = (.5, .75))

,gender,age,salary,family_members,insurance_payments
count,4847.000000,4847.000000,4847.000000,4847.000000,4847.000000
mean,0.498453,31.023932,39895.811223,1.203425,0.152259
std,0.500049,8.487995,9972.952441,1.098664,0.468934
min,0.000000,18.000000,5300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


#### Вывод

Данные изучены, столбцы переименованы для удобства. Числовые форматы приведены к целочисленным. Удалены дубликаты, изучены уникальные значения столбцов датасета

### Пункт 2

<b>Необходимо ответить на вопрос и обосновать решение.</b>

<i> Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.) </i>

<i> a. Изменится. Приведите примеры матриц.</i>

<i>b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.</i>

Свойство обратимой матрицы:

 - Матрицу A называют обратимой, если для нее существует обратная, в противном случае — необратимой.
    
      Из определения следует, что если обратная матрица A<sup>-1</sup> существует, то она квадратная того же порядка, что и A.                Однако не для всякой квадратной матрицы существует обратная. Если определитель матрицы A равен нулю (det A = 0),                то для нее не существует обратной. 
  
Свойства обратных матриц:
 - Обратное значение обратной матрицы A<sup>-1</sup> эквивалентно исходной матрице A: (A<sup>-1</sup>)<sup>-1</sup>=A.
 - Определитель исходной матрицы A соответствует обратному значению детерминанта обратной матрицы A<sup>-1</sup>: |A|=1/|A<sup>-1</sup>|.
 - Матрица, обратная матрице A, умноженной на коэффициент λ≠0, равна значению, полученному при умножении обратной матрицы A<sup>-1</sup> и обратного значения коэффициента λ, то есть (λ×A)<sup>-1</sup>=A<sup>-1</sup>/λ.
 - Обратное значение произведения обратимых матриц A и B с одинаковым числом строк и столбцов будет равно значению, полученному при умножении матриц, обратных исходным, то есть (A×B)<sup>-1</sup>=B<sup>-1</sup>×A<sup>-1</sup>.
 - Обратная матрица транспонированной матрицы эквивалентна транспонированной обратной матрице (A<sup>-1</sup>)<sup>T</sup>=(A<sup>T</sup>)<sup>-1</sup>.  

Обозначения:

 X - матрица признаков (нулевой столбец состоит из единиц)

 y - вектор целевого признака

 Р - матрица, на которую умножаются признаки

 w - вектор весов линейной регрессии 
 
 w* - вектор весов линейной регрессии матрицы полученной домножение исходной на обратимую матрицу

 y^ - предсказания по исходной матрице признаков

 y^* - предсказания по матрице полученной домноженной на обратимую матрицу
 
 
 <b>Исходная матрица признаков Х:</b>
  
  <i>предсказание:</i>
                                   
<center>y = X*w</center>
                                   
  <i>Задача обучения линейной регрессии такая:</i>
    
<center>w = arg min MSE(Xw,y)</center>
<center>w</center>
                                             
   <i>Обучение:</i>
   
<center>w = (X<sup>T</sup>X)<sup>-1</sup>X<sup>T</sup>y</center>





Распишем формулу предсказания:

<center>y^ = X(X<sup>T</sup>X)<sup>-1</sup>X<sup>T</sup>y</center>


Раскроем скобки:
    
<center>y^ = X(X<sup>T</sup>)<sup>-1</sup>X(X)<sup>-1</sup>X<sup>T</sup>y</center>    
    

По свойству единичной матрицы: 
    
<center>y^ = EEy</center> 


<center> y^ = y</center>

<b>новая матрица признаков Х(Р):</b>
  
  <i>предсказание:</i>
                                   
<center>y^* = XPw*</center>
                                   
                                           
  <i>Обуение:</i>
  
   
<center>w* = ((XP)<sup>T</sup>XP)<sup>-1</sup>(XP)<sup>T</sup>y</center>

  <i>Раскоем скобки</i>
  
<center>w* = (X<sup>T</sup>P<sup>T</sup>XP)<sup>-1</sup>X<sup>T</sup>P<sup>T</sup>y</center>



<br>
<center>w* = (X<sup>T</sup>P<sup>T</sup>)<sup>-1</sup>(XP)<sup>-1</sup>X<sup>T</sup>P<sup>T</sup>y</center>
<br>


<center>w* = P<sup>-1</sup> (X<sup>T </sup>X)<sup>-1</sup> 
    (P<sup>T</sup>)<sup>-1</sup>X<sup>T</sup>P<sup>T</sup>y</center>
<br>
  <i>Сокращаем произведение матриц (P<sup>T</sup>)<sup>-1</sup>P<sup>T</sup> по свойству единичной матрицы. Получаем 1, т.е. можно опустить:</i>
<br>
<center>w* = (X<sup>T</sup>X)<sup>-1</sup>P<sup>-1</sup>X<sup>T</sup>y</center>
  


Распишем формулу предсказания:

<center>y^* = XP(X<sup>T</sup>X)<sup>-1</sup>P<sup>-1</sup>X<sup>T</sup>y</center>

<i>Сокращаем произведение матриц P(P)<sup>-1</sup> по свойству единичной матрицы. Получаем 1, т.е. можно опустить:</i>

<center>y^* = X(X<sup>T</sup>X)<sup>-1</sup>X<sup>T</sup>y

Получили формулу для y^ которая полностью совпадает с формулой для y^*

Из этого можно сделать вывод, что в частности значение метрики r2 не изменится

Вывод: при умножении признаков на обратимую матрицу качество линейной регрессии не меняется.

Создадим произвольную матрицу

In [15]:
P = np.random.randint(100, size=(4,4))
print(P)

[[67 60 38 41]
 [68 75 99 36]
 [95  1 67 55]
 [93  5 66 26]]


Проверим на обратимость матрицу.

Умножим матрицу на обратную

In [16]:
np.round(np.dot(P, np.linalg.inv(P)))

array([[ 1., -0., -0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0., -0.,  0.,  1.]])

Получили единичную матрицу, что свидетельствует то матрица  обратимая

### Пункт 3

Выделем признаки и целевой признак:

In [17]:
X = df.drop('insurance_payments', axis=1)
y = df['insurance_payments']

скалируем признаки

In [18]:
scaler = StandardScaler()
scaler.fit(X)



StandardScaler()

In [19]:
X = scaler.transform(X)


По формуле вектора предсказания нам необходимо вычислить веса и подставить в формулу для поиска вектора предсказания. Затем сравнить эти вектора  исходной матрицы и матрицы перемноженной на обратимую матрицу

In [20]:
w = np.linalg.inv(np.dot(X.T, X)).dot(X.T).dot(y)

In [21]:
a = X.dot(w)

In [22]:
X_P = X.dot(P)

In [23]:
w_xp = np.linalg.inv(np.dot(X_P.T, X_P)).dot(X_P.T).dot(y)

In [24]:
a_xp = X_P.dot(w_xp)

In [25]:
print(' Разница векторов предсказаний равна:',np.round(a-a_xp).sum())

 Разница векторов предсказаний равна: 0.0


На практиеской задаче доказали что при умножении признаков на обратимую матрицу качество линейной регрессии не меняется.

### Пункт 4

Создадим модель линейной регрессии и обучим ее на данных датасета

In [26]:
def line_rg (X, y):
    model = LinearRegression()
    model.fit(X, y)
    predictions = model.predict(X)
    return r2_score(y, predictions)

Получили следующую метрику качества r2_score из первоначальной матрицы признаков:

In [27]:
print('метрикa качества r2_score: ', round(line_rg(X,y), 4))

метрикa качества r2_score:  0.4302


Получили следующую метрику качества r2_score из первоначальной матрицы признаков умноженную на обратимую матрицу:

In [28]:
print('метрикa качества r2_score перемноженных матриц: ',round(line_rg(X_P, y),4))

метрикa качества r2_score перемноженных матриц:  0.4302


Проверяем равенство значений метрик

In [29]:
if round(line_rg(X,y), 4) == round(line_rg(X_P, y),4):
    print('Метрики верны')
else:
    print('Ментики не равны')

Метрики верны


In [30]:
#Код ревьюера
def line_rg (X, y):
    model = LinearRegression()
    model.fit(X, y)
    predictions = model.predict(X)
    return r2_score(y, predictions)

In [31]:
#Код ревьюера
print('метрикa качества r2_score: ', round(line_rg(X,y), 4))

метрикa качества r2_score:  0.4302


## Вывод

Были загружены и изучены данные.

Дан ответ на вопрос и обосновано решение:

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? 

  -  Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
  
Дл преобразования данных для решения задачи выбран метод умнажения матрицы признаков на  обратимую матрицу, тк качество линейной регрессии не поменяется.

Запрограммировали этот алгоритм, применив матричные операции. Проверили, что качество линейной регрессии не отличается до и после преобразования.

Применили метрику R2 для оценки качества моделей.